In [12]:
import sys
import pandas as pd
sys.path.append("..")

# news scraping
from src.zinfo.news_scraper import NewsScraper

# clustering
from src.zinfo.article_clustering import get_vectorized_titles
from src.zinfo.article_clustering import cluster_articles

# article selecting
from src.zinfo.article_selector import get_best_article_all_clusters

In [5]:
scraper = NewsScraper('eeeaefaae3c14737bc08e252a6e1991b')

In [6]:
# trending_news = scraper.get_trending_articles_today(5)

scraping articles


In [13]:
trending_news = pd.read_csv("test.csv")

In [9]:
vectors, titles = get_vectorized_titles(trending_news)

In [10]:
clusters = cluster_articles(trending_news)

clustering articles


Exception: Data must be 1-dimensional